In [48]:
import pandas as pd

In [49]:
import os
# Get the current working directory
cwd = os.getcwd()
print(cwd)

# Navigate to the data folder
data_folder = os.path.abspath(os.path.join(cwd, "data"))
print(data_folder)

csv_file = os.path.join(data_folder, "HI-Small_Trans.csv")
print(csv_file)

c:\Users\yrobi\Desktop\Robin World\Data Science - Machine Learning Prep\01 - MLOps\AML-Fraud-Detection\notebook
c:\Users\yrobi\Desktop\Robin World\Data Science - Machine Learning Prep\01 - MLOps\AML-Fraud-Detection\notebook\data
c:\Users\yrobi\Desktop\Robin World\Data Science - Machine Learning Prep\01 - MLOps\AML-Fraud-Detection\notebook\data\HI-Small_Trans.csv


In [50]:
# Load the CSV file into a pandas DataFrame
aml_trans_df = pd.read_csv(csv_file)
# Take small sample of data from the original data
aml_trans_sm_df = aml_trans_df.sample(n=50000, random_state=6)
print(f"Shape of data:  \n No. of records: {aml_trans_sm_df.shape[0]} \n No. features: {aml_trans_sm_df.shape[1]}")
print(f"Data Information: \n{aml_trans_sm_df.info()}")

Shape of data:  
 No. of records: 50000 
 No. features: 11
<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 3507139 to 3159372
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           50000 non-null  object 
 1   From Bank           50000 non-null  int64  
 2   Account             50000 non-null  object 
 3   To Bank             50000 non-null  int64  
 4   Account.1           50000 non-null  object 
 5   Amount Received     50000 non-null  float64
 6   Receiving Currency  50000 non-null  object 
 7   Amount Paid         50000 non-null  float64
 8   Payment Currency    50000 non-null  object 
 9   Payment Format      50000 non-null  object 
 10  Is Laundering       50000 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 4.6+ MB
Data Information: 
None


In [51]:
aml_trans_sm_df.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [52]:
aml_trans_sm_df.columns = aml_trans_sm_df.columns.str.lower().str.replace(' ', '_').str.replace('.', '_')
aml_trans_sm_df.columns 

Index(['timestamp', 'from_bank', 'account', 'to_bank', 'account_1',
       'amount_received', 'receiving_currency', 'amount_paid',
       'payment_currency', 'payment_format', 'is_laundering'],
      dtype='object')

In [53]:
data = aml_trans_sm_df.to_dict(orient="records")
# data

In [54]:
import pymongo

from notebook.key_string import mongoDB_connect_str
connection_string = mongoDB_connect_str

try:
    # Establish connection to MongoDB
    client = pymongo.MongoClient(connection_string)
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")

    # Create or connect to the database
    db_name = "DB_aml_fraud_detector"                      
    db = client[db_name]
    print(f"Connected to database: {db_name}")
    
    # Create or connect to the collection
    collection_name = "collection_aml_fraud_detector"
    collection = db[collection_name]
    print(f"Connected to collection: {collection_name}")
    
    # Inserting all records (data)
    collection.insert_many(data)
    print("Document inserted successfully.")
    
except Exception as e:
    print(e)

# finally:
#     # Close the connection
#     client.close()

Pinged your deployment. You successfully connected to MongoDB!
Connected to database: DB_aml_fraud_detector
Connected to collection: collection_aml_fraud_detector
Document inserted successfully.


In [55]:
records = collection.find().limit(5)
records

In [56]:
for i, j in enumerate(records):
    print(f"{i}: {j}")

0: {'_id': ObjectId('66e1e8d928d0ab00e07b850a'), 'timestamp': '2022/09/07 12:15', 'from_bank': 29, 'account': '80CF063F0', 'to_bank': 235843, 'account_1': '80CFE1EB0', 'amount_received': 386006.86, 'receiving_currency': 'Brazil Real', 'amount_paid': 386006.86, 'payment_currency': 'Brazil Real', 'payment_format': 'Cheque', 'is_laundering': 0}
1: {'_id': ObjectId('66e1e8d928d0ab00e07b850b'), 'timestamp': '2022/09/03 21:15', 'from_bank': 70, 'account': '100428660', 'to_bank': 22732, 'account_1': '80BFEBFF0', 'amount_received': 8638.95, 'receiving_currency': 'US Dollar', 'amount_paid': 8638.95, 'payment_currency': 'US Dollar', 'payment_format': 'Cheque', 'is_laundering': 0}
2: {'_id': ObjectId('66e1e8d928d0ab00e07b850c'), 'timestamp': '2022/09/09 19:22', 'from_bank': 338871, 'account': '8144F97F0', 'to_bank': 15964, 'account_1': '8144FEB20', 'amount_received': 80.84, 'receiving_currency': 'Euro', 'amount_paid': 80.84, 'payment_currency': 'Euro', 'payment_format': 'Credit Card', 'is_launder

In [57]:
df = pd.DataFrame(list(collection.find()))
df.head()

,_id,timestamp,from_bank,account,to_bank,account_1,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering
0,66e1e8d928d0ab00e07b850a,2022/09/07 12:15,29,80CF063F0,235843,80CFE1EB0,386006.86,Brazil Real,386006.86,Brazil Real,Cheque,0
1,66e1e8d928d0ab00e07b850b,2022/09/03 21:15,70,100428660,22732,80BFEBFF0,8638.95,US Dollar,8638.95,US Dollar,Cheque,0
2,66e1e8d928d0ab00e07b850c,2022/09/09 19:22,338871,8144F97F0,15964,8144FEB20,80.84,Euro,80.84,Euro,Credit Card,0
3,66e1e8d928d0ab00e07b850d,2022/09/03 14:11,70,1004286A8,1502,812191060,46480.59,Euro,46480.59,Euro,Cheque,0
4,66e1e8d928d0ab00e07b850e,2022/09/01 14:46,1,803A2CC10,130342,80C982930,118.46,US Dollar,118.46,US Dollar,Cheque,0
